# Synthetic Optical Flow from Fused Lidar


In [1]:
import sys
sys.path.append('/opt/psegs')

## SemanticKITTI

In [2]:
from psegs.exp.semantic_kitti import SemanticKITTISDTable
from psegs.exp.fused_lidar import FusedWorldCloudTableBase

class SemanticKITTIFusedWorldCloudTable(FusedWorldCloudTableBase):
    SRC_SD_TABLE = SemanticKITTISDTable
    
    @classmethod
    def _get_task_lidar_cuboid_rdd(cls, spark, segment_uri):
        seg_rdd = cls.SRC_SD_TABLE.get_segment_datum_rdd(spark, segment_uri)
        
        # SemanticKITTI has no cuboids, so the Fuser algo simply concats the cloud points
        def iter_task_rows(iter_sds):
            from pyspark import Row
            from oarphpy.spark import RowAdapter
            for sd in iter_sds:
                if sd.point_cloud is not None:
                    pc = sd.point_cloud
                    task_id = "%s.%s" % (sd.uri.segment_id, pc.extra['semantic_kitti.scan_id'])
                    yield Row(
                        task_id=task_id,
                        point_clouds=[pc],
                        cuboids=[])
        
        task_rdd = seg_rdd.mapPartitions(iter_task_rows)
        return task_rdd
        


## Start Spark

In [3]:
from psegs.spark import NBSpark
spark = NBSpark.getOrCreate()

2021-02-11 15:52:40,349	oarph 18187 : Using source root /opt/psegs/psegs 
2021-02-11 15:52:40,350	oarph 18187 : Using source root /opt/psegs 
2021-02-11 15:52:40,500	oarph 18187 : Generating egg to /tmp/tmpcb5qm_oj_oarphpy_eggbuild ...
2021-02-11 15:52:40,594	oarph 18187 : ... done.  Egg at /tmp/tmpcb5qm_oj_oarphpy_eggbuild/psegs-0.0.0-py3.8.egg


## Build Fused Lidar Assets

```
docker --context default run -it --name=potree_viewer --rm --net=host -v `pwd`:/shared  jonazpiazu/potree
```

In [4]:
seg_uris = SemanticKITTIFusedWorldCloudTable.get_all_segment_uris()
sd_rdd = SemanticKITTIFusedWorldCloudTable._get_segment_datum_rdd_or_df(spark, seg_uris[1])
print(sd_rdd.count())

2021-02-11 15:52:43,852	ps   18187 : Found Sequence 00 with 4541 scans
2021-02-11 15:52:43,856	ps   18187 : Found Sequence 01 with 1101 scans
2021-02-11 15:52:43,861	ps   18187 : Found Sequence 02 with 4661 scans
2021-02-11 15:52:43,864	ps   18187 : Found Sequence 03 with 801 scans
2021-02-11 15:52:43,866	ps   18187 : Found Sequence 04 with 271 scans
2021-02-11 15:52:43,871	ps   18187 : Found Sequence 05 with 2761 scans
2021-02-11 15:52:43,874	ps   18187 : Found Sequence 06 with 1101 scans
2021-02-11 15:52:43,877	ps   18187 : Found Sequence 07 with 1101 scans
2021-02-11 15:52:43,880	ps   18187 : Found Sequence 09 with 1591 scans
2021-02-11 15:52:43,884	ps   18187 : Found Sequence 10 with 1201 scans
2021-02-11 15:52:43,884	ps   18187 : Found 19130 total scans
2021-02-11 15:52:43,887	ps   18187 : Filtering to only 1 segments
2021-02-11 15:52:43,888	ps   18187 : SemanticKITTIFusedWorldCloudTable building fused world clouds ...
2021-02-11 15:52:43,889	ps   18187 : ... have 1 segments to fu

1
